# 5 - Regresión lineal y logística

## Preparación del entorno


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss
from sklearn.datasets import load_iris

# Reproducibilidad
np.random.seed(42)
pd.set_option("display.precision", 3)

**Dataset 1**: `Iris`

Se utilizará la variable numérica `petal length` como variable objetivo para realizar técnicas de regresión lineal.

In [ ]:
iris = load_iris(as_frame=True)
df_iris = pd.concat([iris.data, pd.Series(iris.target, name="target")], axis=1)
df_iris.rename(columns=lambda c: c.replace('(cm)', '').strip(), inplace=True)
df_iris.head()

**Dataset 2**: `Titanic`

Se utilizará este dataset para predecir la  supervivencia (`survived`) mediante regresión logística.

In [ ]:
# Carga de dataset
df_titanic = sns.load_dataset('titanic')
df_titanic.head(3)

In [ ]:
# Selección mínima de columnas y limpieza simple
cols = ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
df_titanic = df_titanic[cols].copy()
df_titanic = df_titanic.dropna(subset=cols)

In [ ]:
# One-Hot básico para 'sex' y 'embarked'
# N-1 columnas generadas
df_titanic = pd.get_dummies(df_titanic, columns=['sex', 'embarked'], drop_first=True)
print('Dimensiones:', df_titanic.shape)
df_titanic.head(3)

## 1. Regresión lineal simple

* **Característica(s)**: `sepal length`
* **Variable objetivo**: `petal length`

In [ ]:
# Dividir el dataset de origen en X (conjunto de características) e y (variable objetivo)
y = df_iris['petal length']
X = df_iris[['sepal length']]

In [ ]:
# Realizar el ajuste/entrenamiento del modelo de regresión lineal simple
lin1 = LinearRegression()
lin1.fit(X, y)

In [ ]:
# Mostrar función resultante
b0 = float(lin1.intercept_)
b1 = float(lin1.coef_[0])
print(f"Modelo de regresión lineal simple:\ny = {b0:.3f} (b0) + {b1:.3f} (b1) * sepal_length")

In [ ]:
# Gráfico de dispersión + recta
x_grid = np.linspace(X.min()[0], X.max()[0], 100).reshape(-1, 1)
y_hat_grid = lin1.predict(x_grid)

plt.figure(figsize=(6,4))
plt.scatter(X, y)
plt.plot(x_grid, y_hat_grid)
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.title('Regresión lineal simple')
plt.show()

In [ ]:
# Cálculo de métricas para descripción del rendimiento
y_hat = lin1.predict(X)

mse = mean_squared_error(y, y_hat)
mae = mean_absolute_error(y, y_hat)
r2  = r2_score(y, y_hat)

print(f"Error Medio Cuadrático - MSE = {mse:.3f}")
print(f"Error Absoluto Medio - MAE = {mae:.3f}")
print(f"R^2 = {r2:.3f}\t(descripción, sin validación)")

In [ ]:
# Visualización de los residuos
resid = y - y_hat

plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.scatter(y_hat, resid)
plt.axhline(0)
plt.xlabel('Predicción (y_hat)')
plt.ylabel('Residuo (y - y_hat)')
plt.title('Residuos vs predicción')

plt.subplot(1, 2, 2)
plt.hist(resid, bins=20)
plt.xlabel('Residuo')
plt.ylabel('Frecuencia')
plt.title('Distribución de residuos')

plt.tight_layout()
plt.show()

## 2. Regresión lineal multivariable

* **Característica(s)**: `sepal length`, `sepal width` y `petal width`
* **Variable objetivo**: `petal length`

In [ ]:
# Dividir el dataset de origen en Xm (conjunto de características) e ym (variable objetivo)
features = ['sepal length', 'sepal width', 'petal width']
Xm = df_iris[features]
ym = df_iris['petal length']

In [ ]:
# Realizar el ajuste/entrenamiento del modelo de regresión lineal multivariable
linm = LinearRegression()
linm.fit(Xm, ym)

In [ ]:
# Calcular función resultante
coefs = pd.DataFrame({'feature': features, 'coef': linm.coef_})

function_str = f"Modelo de regresión lineal multivariable:\n\ny = {linm.intercept_:.3f} (b0)"
for i, row in coefs.iterrows():
    function_str += f"\n + {row['coef']:.3f} (b{i}) * {row['feature']} (x{i})"
print(function_str)

In [ ]:
# Cálculo de métricas para descripción del rendimiento
yhat_m = linm.predict(Xm)

mse_m = mean_squared_error(ym, yhat_m)
mae_m = mean_absolute_error(ym, yhat_m)
r2_m  = r2_score(ym, yhat_m)

print(f"Error Medio Cuadrático - MSE = {mse_m:.3f}")
print(f"Error Absoluto Medio - MAE = {mae_m:.3f}")
print(f"R^2 = {r2_m:.3f}\t(descripción, sin validación)")

In [ ]:
# Visualización de los residuos
resid_m = ym - yhat_m

plt.figure(figsize=(6,4))
plt.scatter(yhat_m, resid_m)
plt.axhline(0)
plt.xlabel('Predicción (y_hat)')
plt.ylabel('Residuo')
plt.title('Residuos vs predicción')
plt.show()

## 3. Regresión polinómica

Si se sospecha que ocurre una dispersión con **curvatura**, se deben crear **términos no lineales**.

Aquí se ilustra una polinómica de **grado 2** con `sepal length` y `petal width`.

In [ ]:
# Dividir el dataset de origen en X (conjunto de características) e y (variable objetivo)
# + Realizar el ajuste/entrenamiento del modelo de regresión "lineal" polinómico
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(df_iris[['sepal length', 'petal width']])
lin_poly = LinearRegression().fit(X_poly, df_iris['petal length'])

In [ ]:
# Cálculo de métricas para descripción del rendimiento
yhat_p = lin_poly.predict(X_poly)

mse_p = mean_squared_error(df_iris['petal length'], yhat_p)
mae_p = mean_absolute_error(df_iris['petal length'], yhat_p)
r2_p  = r2_score(df_iris['petal length'], yhat_p)

print(f"Error Medio Cuadrático - MSE = {mse:.3f}")
print(f"Error Absoluto Medio - MAE = {mae:.3f}")
print(f"R^2 = {r2:.3f}\t(descripción, sin validación)")

In [ ]:
# Visualización de los residuos
resid_p = df_iris['petal length'] - yhat_p
plt.figure(figsize=(6,4))
plt.scatter(yhat_p, resid_p)
plt.axhline(0)
plt.xlabel('Predicción (y_hat)')
plt.ylabel('Residuo')
plt.title('Residuos vs predicción')
plt.show()

## 4. Regresión logística

* **Dataset**: `Titanic`
* **Variable objetivo**: `survived`


In [ ]:
# Dividir el dataset de origen en X_clf (conjunto de características) e y_clf (variable objetivo)
y_clf = df_titanic['survived'].astype(int)
X_clf = df_titanic.drop(columns=['survived'])

In [ ]:
# Realizar el ajuste/entrenamiento del modelo de regresión logística
logreg = LogisticRegression(max_iter=1000, solver='lbfgs')
logreg.fit(X_clf, y_clf)

In [ ]:
# Muestra de los resultados (en probabilidades)
proba = logreg.predict_proba(X_clf)[:, 1]
print(f"Probabilidad [0,1] de target=1 por registro (5 primeros):\n{proba[:5]}")

In [ ]:
# Cálculo del logloss para descripción del rendimiento
ll = log_loss(y_clf, proba)
print(f'LogLoss (solo descriptivo): {ll:.4f}')

**Interpretación de coeficientes como Odds Ratio**

En regresión logística, los coeficientes ($\beta$) se interpretan más fácilmente transformándolos en **Odds Ratios** (OR) mediante la función exponencial: $OR = e^{\beta}$.

El Odds Ratio representa el factor por el cual las *odds* de que ocurra el evento (en este caso, `survived=1`) cambian por cada unidad de aumento en la variable predictora, manteniendo el resto de variables constantes.

*   **OR > 1**: Indica que un aumento en la característica está asociado con un **aumento** en las *odds* del evento.
*   **OR < 1**: Indica que un aumento en la característica está asociado con una **disminución** en las *odds* del evento.
*   **OR = 1**: Indica que la característica **no tiene efecto** en las *odds* del evento.

Un OR de 2.0 significa que las odds del evento se duplican por cada unidad de aumento en la característica. Un OR de 0.5 significa que las odds del evento se reducen a la mitad por cada unidad de aumento en la característica.

In [ ]:
# Interpretación básica de coeficientes como odds ratios
or_table = pd.DataFrame({
    'feature': X_clf.columns,
    'coef': logreg.coef_.ravel(),
})
or_table['odds_ratio'] = np.exp(or_table['coef'])
or_table.sort_values('odds_ratio', ascending=False, inplace=True)
or_table

In [ ]:
# Visualización histograma de probabilidades por clase real
plt.figure(figsize=(6,4))
plt.hist(proba[y_clf==1], bins=20, alpha=0.7, label='y=1')
plt.hist(proba[y_clf==0], bins=20, alpha=0.7, label='y=0')
plt.xlabel('Probabilidad predicha de Y=1')
plt.ylabel('Frecuencia')
plt.title('Distribución de probabilidades por clase real (visual)')
plt.legend()
plt.show()

## Ejercicio

1. Elige otro dataset real y **replica una regresión lineal y una logística**.
2. **Experimenta** con los parámetros de las funciones y clases utilizadas, y observa sus efectos:

    2.1. **Regresión lineal y polinómica**:
    * `fit_intercept`
    * `PolynomialFeatures`

    2.2. **Regresión logística**:
    * `penalty`
    * `solver`